In [61]:
import os
import pandas as pd
import numpy  as np

In [26]:
data=r"C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_dataset"
no_tumor=r"C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_dataset\no"
yes_tumor=r"C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_dataset\yes"

In [27]:
dirlist=[no_tumor,yes_tumor]
classes=['No','Yes']
filepaths=[]
labels=[]

In [28]:
for i,j in zip(dirlist,classes):
    filelists=os.listdir(i)
    for f in filelists:
        filepaths.append(os.path.join(i,f))
        labels.append(j)

In [29]:
files=pd.Series(filepaths,name="filepaths")
labels_s=pd.Series(labels,name="labels")
df=pd.concat([files,labels_s],axis=1)

In [30]:
df

,filepaths,labels
0,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,No
1,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,No
2,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,No
3,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,No
4,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,No
...,...,...
248,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,Yes
249,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,Yes
250,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,Yes
251,C:\Users\VAMSI\OneDrive\Desktop\brain_tumor_da...,Yes


In [32]:
df = df.sample(frac = 1)

In [33]:
df=df.reset_index()

In [34]:
df.drop('index',inplace=True,axis=1)

In [35]:
df['labels']=pd.get_dummies(df['labels'])

In [36]:
from PIL import Image

In [40]:
img=Image.open(df['filepaths'][1])

In [41]:
img.show()

In [51]:
widths=[]
heights=[]
for idx,rows in df.iterrows():
    path=rows['filepaths']
    img=Image.open(path)
    width,height=img.size
    widths.append(width)
    heights.append(height)
avg_width=int(sum(widths)/len(widths))
avg_height=int(sum(heights)/len(heights))
    
print(avg_width)
print(avg_height)
    

354
386


In [54]:
from keras.preprocessing.image import load_img
def load_resize_color_image(path):
    # load image and resize to 300x300
    image = load_img(path,target_size=(300,300))
    return image

In [55]:
image_list=[]
cancer_list=[]
from keras.preprocessing.image import img_to_array
for idx,rows in df.iterrows():
    path=rows['filepaths']
    cancer=rows['labels']
    image= load_resize_color_image(path)
    image_array = img_to_array(image)   
    image_list.append(image_array)
    cancer_list.append(cancer)

In [56]:
from sklearn.utils import shuffle
image_list, cancer_list = shuffle(image_list, cancer_list)

In [62]:
x=np.array(image_list)
y=np.array(cancer_list)

In [63]:
x.shape

(253, 300, 300, 3)

In [68]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


In [69]:
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import  Flatten
from keras.layers import Input

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

In [71]:
X_train = X_train/255
X_test = X_test/255

In [66]:
epochs=10
batch_size=15
input_size=(300, 300, 3)

In [70]:
from keras.models import Sequential
from keras.layers import Dense

In [76]:
model=Sequential()
model.add(Input(shape=(300,300,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [77]:
model.compile(loss="binary_crossentropy",
              optimizer='Adam',
              metrics=['accuracy'])

In [78]:
hist = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 202 samples, validate on 51 samples
Epoch 1/10
202/202 [==============================] - ETA: 0s - loss: 1.5951 - acc: 0.6337

C:\Users\VAMSI\anaconda3\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


202/202 [==============================] - 33s 164ms/sample - loss: 1.5951 - acc: 0.6337 - val_loss: 0.5501 - val_acc: 0.7451
Epoch 2/10
202/202 [==============================] - 30s 149ms/sample - loss: 0.5097 - acc: 0.8020 - val_loss: 0.5199 - val_acc: 0.7843
Epoch 3/10
202/202 [==============================] - 30s 149ms/sample - loss: 0.3443 - acc: 0.8663 - val_loss: 0.4534 - val_acc: 0.8431
Epoch 4/10
202/202 [==============================] - 30s 150ms/sample - loss: 0.1194 - acc: 0.9752 - val_loss: 0.8987 - val_acc: 0.6667
Epoch 5/10
202/202 [==============================] - 30s 150ms/sample - loss: 0.0916 - acc: 0.9505 - val_loss: 0.6545 - val_acc: 0.8627
Epoch 6/10
202/202 [==============================] - 30s 150ms/sample - loss: 0.0203 - acc: 0.9950 - val_loss: 1.2702 - val_acc: 0.8431
Epoch 7/10
202/202 [==============================] - 31s 151ms/sample - loss: 0.0090 - acc: 1.0000 - val_loss: 0.9246 - val_acc: 0.8431
Epoch 8/10
202/202 [==============================] 

In [79]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0511988448161704
Test accuracy: 0.88235295
